In [1]:
import copy

import numpy as np
from numba import njit

from ase.cluster import Octahedron
from ase.io import read

from npl.core import Nanoparticle
from npl.surrogate_energy_models import TopologicalEnergyModel
from npl.descriptors import Topologies

In [2]:
#system = read("/home/riccardo/Scientific_Projects/PtAu_bigNps/300K/10_perc/run2/glo002.xyz")
system = Octahedron('Pt', 2)
for f in np.random.choice(len(system), 2, replace=False):
    system[f].symbol = 'Y'


system = Nanoparticle.from_atoms(system)
Topologies(['Pt', 'Y']).get_feature_vector(system)

#top = TopologicalEnergyModel(['Pt', 'Y'])
#top.get_training_set([system])

array([ 2, 12, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [3]:
from npl.core import StructureContainer

system = Octahedron('Pt', 2)
structure_container = StructureContainer(Topologies(['Pt', 'Y']))
structure_container.add_structure(system)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [10]:
from typing import List

def ciao() -> List[np.array]:
    return [np.array([1]), np.array([2])]

ciao()

[array([1]), array([2])]

In [2]:
from npl.surrogate_energy_models import topological_energy_model

IndentationError: expected an indented block (topological_energy_model.py, line 19)

In [4]:
%timeit exchange_operator.perform_operation(system)

164 µs ± 756 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
%timeit top.create(system)

88.7 µs ± 471 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [2]:
exchange_operator.perform_operation(system)
top.create(system)
system.get_numbers_from_occupation_matrix()

[39, 39, 78, 78, 78, 78]

In [3]:
exchange_operator.revert_operation(system)
system.get_numbers_from_occupation_matrix()

[78, 78, 78, 39, 78, 39]

In [13]:
np.array(old_indices) - np.array(new_old_indices)

array([0, 0, 0, ..., 0, 0, 0])

In [4]:
exchange_operator = ExchangeOperator(system, 0.5)
for _ in range(1000):
    exchange_operator.perform_operation(system)
system.OM.T[1]
#new_indices = system.get_numbers_from_occupation_matrix()

array([1, 0, 0], dtype=int8)

In [12]:
system.OM.T[1]

array([0, 0, 1], dtype=int8)

In [8]:
%timeit exchange_operator.perform_operation(system)

153 µs ± 450 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
exchange_operator = ExchangeOperator(system, 0.5)
exchange_operator.perform_operation(system)
exchange_operator.revert_operation(system)

In [3]:
system.get_number_index(78)

NameError: 

In [3]:
np.unique([5,24,15])

array([ 5, 15, 24])

In [18]:

index = np.where(system.get_unique_numbers() == 78)[0][0]
index

0

In [2]:
system.get_occupation_indices_by_symbol(78)

array([   2,   25,   26, ..., 1460, 1461, 1462])

In [3]:
for atomic_number in system.get_unique_numbers():
    print(system.get_occupation_indices_by_symbol(atomic_number))

[0 0 0 ... 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
exchange_operator.indices_by_element

NameError: name 'exchange_operator' is not defined

In [6]:
exchange_operator.get_swap_indices()

([(0, 0)], [(78, 79)])

In [4]:
system.get_occupation_indices_by_symbol('Pt')

IndexError: index 2 is out of bounds for axis 0 with size 2

In [9]:
from itertools import combinations_with_replacement

for f in combinations_with_replacement(range(len(system.OM)),2):
    print(f)

(0, 0)
(0, 1)
(0, 2)
(1, 1)
(1, 2)
(2, 2)


In [5]:
system.atom_features['TOP'].sum(axis=0)

array([    0,     0,    12, 13488,  1116,   936,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0])

In [26]:
np.zeros(5, dtype=np.int8)

array([0, 0, 0, 0, 0], dtype=int8)

In [14]:
np.unique(system.symbols)

array(['Au', 'Ni', 'Pt'], dtype='<U2')

In [16]:
top.bond_types

[(28, 28), (28, 78), (28, 79), (78, 78), (78, 79), (79, 79)]

In [15]:
top.bond_type_per_element

{28: [0, 1, 2], 78: [1, 3, 4], 79: [2, 4, 5]}

In [2]:
from npl.descriptors import Topologies

top = Topologies(np.unique(system.symbols))
top.create(system)

0


TypeError: Cannot interpret '0' as a data type

In [ ]:
from itertools import combinations_with_replacement

atomic_numbers = np.unique(system.numbers)

bonds_types = [x for x in combinations_with_replacement(atomic_numbers, 2)]
bond_types_order = {x : [] for x in atomic_numbers}

for atomic_number in atomic_numbers:
    for bond_type in bonds_types:
        if atomic_number in bond_type:
            bond_types_order[atomic_number].append(bonds_types.index(bond_type))



In [4]:
def compute_atom_features(system, i):
    key = system[i].number
    vector = np.zeros(len(bonds_types))
    for l, bond_type in enumerate(bond_types_order[key]):
        vector[bond_type] += np.dot(CM[:,i], OM[l])
    
    return vector

In [5]:
atomic_numbers

array([28, 78, 79])

In [71]:
bond_types_order

{28: [0, 1, 2], 78: [1, 3, 4], 79: [2, 4, 5]}

In [7]:
compute_atom_features(system, 30)

array([0., 0., 0., 0., 5., 7.])

In [68]:
system[30]

Atom('Au', [47.8574, 31.9049, 51.8455], tag=0, momentum=[0.0, 0.0, 0.0], mass=196.966569, magmom=0.0, charge=0.0, index=30)

In [57]:
bond_types_order

{28: [0, 1, 2], 78: [1, 3, 4], 79: [2, 4, 5]}

In [62]:
vector = np.zeros(len(bonds_types))
for i in range(len(system)):
    key = system[i].number
    
    for l, bond_type in enumerate(bond_types_order[key]):
        vector[bond_type] += np.dot(CM[i], OM[l])


In [34]:
class ExchangeMove:
    def __init__(self, system):
        self.occupation_matrix = system._occupation_matrix
        self.a = np.where(self.occupation_matrix==1)[1]
        self.b = np.where(self.occupation_matrix==0)[1]

    def exchange_move(self):
        swap1 = np.random.choice(self.a)
        swap2 = np.random.choice(self.b)

        self.a[swap1], self.b[swap2] = self.a[swap2], self.b[swap1]

        self.occupation_matrix[0,swap1], self.occupation_matrix[1,swap1] = self.occupation_matrix[1,swap1], self.occupation_matrix[0,swap1]
        self.occupation_matrix[0,swap2], self.occupation_matrix[1,swap2] = self.occupation_matrix[1,swap2], self.occupation_matrix[0,swap2]

ex = ExchangeMove(system)
ex.exchange_move()

In [39]:
for _ in range(20000):
    ex.exchange_move()
    bond_matrix = system.get_bond_matrix()
    get_top_descriptors(system._occupation_matrix, bond_matrix)

In [20]:
np.where(system._occupation_matrix==1)[1]

array([   0,    1,    3, ..., 1460, 1461, 1462])

In [63]:
@njit
def dio(ciao, ciao2):
    ciao.dot(ciao2) 

%timeit dio(system._connectivity_matrix, system._occupation_matrix.T)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
- Resolution failure for literal arguments:
No implementation of function Function(<function array_dot at 0x7fac07f6e940>) found for signature:

 >>> array_dot(array(int16, 2d, C), array(float64, 2d, F))

There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'array_dot': File: numba/np/arrayobj.py: Line 5526.
        With argument(s): '(array(int16, 2d, C), array(float64, 2d, F))':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<function dot at 0x7fac70599790>) found for signature:

        >>> dot(array(int16, 2d, C), array(float64, 2d, F))

       There are 4 candidate implementations:
             - Of which 4 did not match due to:
             Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: 

In [55]:
system._connectivity_matrix * system._occupation_matrix[1]

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [59]:
system._connectivity_matrix.dot(system._occupation_matrix.T)

array([[6., 3.],
       [4., 5.],
       [4., 5.],
       ...,
       [5., 4.],
       [7., 2.],
       [7., 2.]])